In [1]:
#Set dependencies
import pandas as pd
import os
import numpy as np

sensors = ['42043','42044','42047','8764314','8770613', 'TABS-X']
months = ['June 2017','July 2017','August 2017','Sept 2017']
monthsf = ['0617','0717','0817','0917']

file1 = []
file2 = []
file3 = []
file4 = []
file5 = []

#naming csv file for all resources
for i in sensors:
    for j in months:
        file1.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} air_pressure.csv")
        file2.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} air_temperature.csv")
        file3.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} sea_water_practical_salinity.csv")
        file4.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} sea_water_temperature.csv")
        file5.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} winds.csv")
        
print(len(file1),len(file2),len(file3),len(file4),len(file5))

24 24 24 24 24


In [2]:
# Read all csv files
files = []
for i in range(len(sensors)*len(months)):
        print(f'Reading csvfile {i}')
        df1 = pd.read_csv(file1[i], parse_dates=['date'])
        df2 = pd.read_csv(file2[i], parse_dates=['date'])
        df3 = pd.read_csv(file3[i], parse_dates=['date'])
        df4 = pd.read_csv(file4[i], parse_dates=['date'])
        df5 = pd.read_csv(file5[i], parse_dates=['date'])
        dfa = [df1,df2,df3,df4,df5]
        files.append(dfa)
        
dfx = pd.DataFrame(files) 
dfx.shape

Reading csvfile 0
Reading csvfile 1
Reading csvfile 2
Reading csvfile 3
Reading csvfile 4
Reading csvfile 5
Reading csvfile 6
Reading csvfile 7
Reading csvfile 8
Reading csvfile 9
Reading csvfile 10
Reading csvfile 11
Reading csvfile 12
Reading csvfile 13
Reading csvfile 14
Reading csvfile 15
Reading csvfile 16
Reading csvfile 17
Reading csvfile 18
Reading csvfile 19
Reading csvfile 20
Reading csvfile 21
Reading csvfile 22
Reading csvfile 23


(24, 5)

In [3]:
#Drop any unwanted columns and analyze the difference in date frequency PER CSV FILE
delta_date = []
for i in range(len(sensors)*len(months)):    
    df1_diff = dfx[0][i].assign(diff=dfx[0][i].groupby('latitude')['date'].diff()).dropna()
    delta_date.append(np.mean(df1_diff["diff"]))

    df2_diff = dfx[1][i].assign(diff=dfx[1][i].groupby('latitude')['date'].diff()).dropna()
    delta_date.append(np.mean(df2_diff["diff"]))

    df3_diff = dfx[2][i].assign(diff=dfx[2][i].groupby('latitude')['date'].diff()).dropna()
    delta_date.append(np.mean(df3_diff["diff"]))

    df4_diff = dfx[3][i].assign(diff=dfx[3][i].groupby('latitude')['date'].diff()).dropna()
    delta_date.append(np.mean(df4_diff["diff"]))

    df5_diff = dfx[4][i].assign(diff=dfx[4][i].groupby('latitude')['date'].diff()).dropna()
    delta_date.append(np.mean(df5_diff["diff"]))

    print(max(delta_date))
    print()

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 00:45:02.362204

0 days 01:09:28.115942

0 days 01:09:28.115942

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213

0 days 01:10:19.905213



In [4]:
# Merge all the dataframes into one complete using concat function then export to CSV
c = 0
for i in range(len(sensors)):
    for j in range(len(months)):
        print(f'Exporting cleaned csvfile {c}')
        df_all = [dfx[0][c],dfx[1][c],dfx[2][c],dfx[3][c],dfx[4][c]]
        df = pd.concat(df_all, join='outer', axis=1).dropna()
        df = df.loc[:,~df.columns.duplicated()]
        df_clean = df[['sensor','date', 'latitude', 'longitude',
           'air_pressure(mBar)', 'air_temperature',   'sea_water_practical_salinity', 'sea_water_temperature', 'wind_speed']]

        #Set the file path to export city data
        csvFile = os.path.join("../data/2017",f"{sensors[i]}-{monthsf[j]}.csv")

        #Export clean datframe to csv file
        df_clean.to_csv(csvFile)
        
        #Increment counter by 1
        c+=1

Exporting cleaned csvfile 0
Exporting cleaned csvfile 1
Exporting cleaned csvfile 2
Exporting cleaned csvfile 3
Exporting cleaned csvfile 4
Exporting cleaned csvfile 5
Exporting cleaned csvfile 6
Exporting cleaned csvfile 7
Exporting cleaned csvfile 8
Exporting cleaned csvfile 9
Exporting cleaned csvfile 10
Exporting cleaned csvfile 11
Exporting cleaned csvfile 12
Exporting cleaned csvfile 13
Exporting cleaned csvfile 14
Exporting cleaned csvfile 15
Exporting cleaned csvfile 16
Exporting cleaned csvfile 17
Exporting cleaned csvfile 18
Exporting cleaned csvfile 19
Exporting cleaned csvfile 20
Exporting cleaned csvfile 21
Exporting cleaned csvfile 22
Exporting cleaned csvfile 23
